# Import Libraries


In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from IPython import display

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [54]:
# params_cfg = {
#     "action"  : "main_feat01",  
#     "seed"    : 42, # Set random seed
#     "exp_dir" : os.path.abspath('../../exps'),
#     'exp_name': 'featbase_18102025',
#     "data_dir": os.path.abspath("../../data/titanic"),
#     "verbose" : True,
# }
params_cfg = {
    "action"   : "train_feat01",  
    "feat_path": "../../exps/featbase_18102025/data.npz",
    "seed"    : 42, # Set random seed
    "exp_dir" : os.path.abspath('../../exps'),
    'exp_name': 'trainbase_18102025',
    "data_dir": os.path.abspath("../../../data"),
    "verbose" : True,
}
params_cfg.update(**{
    "save_dir": os.path.abspath(f'{params_cfg["exp_dir"]}/{params_cfg["exp_name"]}')
})

for v in params_cfg:
    print(f'+ {v}: {params_cfg[v]}')

globals().update(**params_cfg)

+ action: train_feat01
+ feat_path: ../../exps/featbase_18102025/data.npz
+ seed: 42
+ exp_dir: c:\Users\LOQ\Downloads\tmp\pocess_challenge1_titanic\process\exps
+ exp_name: trainbase_18102025
+ data_dir: c:\Users\LOQ\Downloads\tmp\pocess_challenge1_titanic\data
+ verbose: True
+ save_dir: c:\Users\LOQ\Downloads\tmp\pocess_challenge1_titanic\process\exps\trainbase_18102025


# Data Load


In [55]:
df_train = pd.read_csv(f'{data_dir}/train.csv')
df_test = pd.read_csv(f'{data_dir}/test.csv')

if params_cfg["verbose"]:
    print("-"*10, "information", "-"*10)
    print(f'train-col: {set(df_train.columns)}')
    print(f'test-col: {set(df_test.columns)}')
    print("Union:", set(df_train.columns).intersection(set(df_test.columns)))
    print("Difference:", set(df_train.columns).difference(set(df_test.columns)))

---------- information ----------
train-col: {'Survived', 'Parch', 'Pclass', 'Fare', 'Ticket', 'Cabin', 'Name', 'Embarked', 'SibSp', 'PassengerId', 'Sex', 'Age'}
test-col: {'Parch', 'Pclass', 'Fare', 'Ticket', 'Cabin', 'Name', 'Embarked', 'SibSp', 'PassengerId', 'Sex', 'Age'}
Union: {'Parch', 'Fare', 'Pclass', 'Sex', 'Ticket', 'Name', 'Embarked', 'SibSp', 'PassengerId', 'Cabin', 'Age'}
Difference: {'Survived'}


# Preprocessing


## Check missing value


In [56]:
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [57]:
df_test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## Sex: giới tính


In [58]:
np.unique(df_train['Sex'].values)

array(['female', 'male'], dtype=object)

Nhận xét:

- Cột Sex là thuộc tính Category với 2 giới trị 'male' và 'female'
- Không có missing value
  => Ta sẽ mã hóa cột Sex dưới dạng số: "male" - 1 & "female" - 0


In [59]:
cls_sex = {'male': 1, 'female': 0}

In [60]:
df_output = pd.DataFrame()
df_output['Sex'] = df_train['Sex'].apply(lambda x: cls_sex[x])

## Age: Tuổi


In [61]:
np.unique(df_train['Age'].values)

array([ 0.42,  0.67,  0.75,  0.83,  0.92,  1.  ,  2.  ,  3.  ,  4.  ,
        5.  ,  6.  ,  7.  ,  8.  ,  9.  , 10.  , 11.  , 12.  , 13.  ,
       14.  , 14.5 , 15.  , 16.  , 17.  , 18.  , 19.  , 20.  , 20.5 ,
       21.  , 22.  , 23.  , 23.5 , 24.  , 24.5 , 25.  , 26.  , 27.  ,
       28.  , 28.5 , 29.  , 30.  , 30.5 , 31.  , 32.  , 32.5 , 33.  ,
       34.  , 34.5 , 35.  , 36.  , 36.5 , 37.  , 38.  , 39.  , 40.  ,
       40.5 , 41.  , 42.  , 43.  , 44.  , 45.  , 45.5 , 46.  , 47.  ,
       48.  , 49.  , 50.  , 51.  , 52.  , 53.  , 54.  , 55.  , 55.5 ,
       56.  , 57.  , 58.  , 59.  , 60.  , 61.  , 62.  , 63.  , 64.  ,
       65.  , 66.  , 70.  , 70.5 , 71.  , 74.  , 80.  ,   nan])

Nhận xét:

- Cột Age là thuộc tính dạng số
- Có missing value
  => Ta sẽ fill missing value đơn giản bằng giá trị trung bình mean()


In [62]:
df_output["Age"] = df_train["Age"].fillna(df_train["Age"].mean())

## Fare: Giá vé


In [63]:
np.unique(df_train['Fare'].values)

array([  0.    ,   4.0125,   5.    ,   6.2375,   6.4375,   6.45  ,
         6.4958,   6.75  ,   6.8583,   6.95  ,   6.975 ,   7.0458,
         7.05  ,   7.0542,   7.125 ,   7.1417,   7.225 ,   7.2292,
         7.25  ,   7.3125,   7.4958,   7.5208,   7.55  ,   7.6292,
         7.65  ,   7.725 ,   7.7292,   7.7333,   7.7375,   7.7417,
         7.75  ,   7.775 ,   7.7875,   7.7958,   7.8   ,   7.8292,
         7.8542,   7.875 ,   7.8792,   7.8875,   7.8958,   7.925 ,
         8.0292,   8.05  ,   8.1125,   8.1375,   8.1583,   8.3   ,
         8.3625,   8.4042,   8.4333,   8.4583,   8.5167,   8.6542,
         8.6625,   8.6833,   8.7125,   8.85  ,   9.    ,   9.2167,
         9.225 ,   9.35  ,   9.475 ,   9.4833,   9.5   ,   9.5875,
         9.825 ,   9.8375,   9.8417,   9.8458,  10.1708,  10.4625,
        10.5   ,  10.5167,  11.1333,  11.2417,  11.5   ,  12.    ,
        12.275 ,  12.2875,  12.35  ,  12.475 ,  12.525 ,  12.65  ,
        12.875 ,  13.    ,  13.4167,  13.5   ,  13.7917,  13.8

Nhận xét:

- Cột Fare là thuộc tính dạng số thực biểu thị giá vé của hàng khách
- Có missing value
  => Ta sẽ tạm thời giữ nguyên cột này


## Pclass: Hạng


In [64]:
np.unique(df_train['Pclass'].values)

array([1, 2, 3])

Nhận xét:

- Cột Pclass là thuộc tính Category gồm các giá trị số 1 2 3
- Không có missing value
  => Ta sẽ tạm thời giữ nguyên cột này


## Cabin: Tên cabin


In [65]:
np.unique(df_train['Cabin'].dropna().values)

array(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31',
       'A32', 'A34', 'A36', 'A5', 'A6', 'A7', 'B101', 'B102', 'B18',
       'B19', 'B20', 'B22', 'B28', 'B3', 'B30', 'B35', 'B37', 'B38',
       'B39', 'B4', 'B41', 'B42', 'B49', 'B5', 'B50', 'B51 B53 B55',
       'B57 B59 B63 B66', 'B58 B60', 'B69', 'B71', 'B73', 'B77', 'B78',
       'B79', 'B80', 'B82 B84', 'B86', 'B94', 'B96 B98', 'C101', 'C103',
       'C104', 'C106', 'C110', 'C111', 'C118', 'C123', 'C124', 'C125',
       'C126', 'C128', 'C148', 'C2', 'C22 C26', 'C23 C25 C27', 'C30',
       'C32', 'C45', 'C46', 'C47', 'C49', 'C50', 'C52', 'C54', 'C62 C64',
       'C65', 'C68', 'C7', 'C70', 'C78', 'C82', 'C83', 'C85', 'C86',
       'C87', 'C90', 'C91', 'C92', 'C93', 'C95', 'C99', 'D', 'D10 D12',
       'D11', 'D15', 'D17', 'D19', 'D20', 'D21', 'D26', 'D28', 'D30',
       'D33', 'D35', 'D36', 'D37', 'D45', 'D46', 'D47', 'D48', 'D49',
       'D50', 'D56', 'D6', 'D7', 'D9', 'E10', 'E101', 'E12', 'E121',
       'E17',

Nhận xét:

- Cột Cabin là thuộc tính dạng String
- Có rất nhiều missing value
  => Ta sẽ drop cột này khỏi tập feature vì khó xử lý


## Embarked: Cảng


In [66]:
np.unique(df_train['Embarked'].dropna().values)

array(['C', 'Q', 'S'], dtype=object)

Nhận xét:

- Cột Embarked là thuộc tính Category gồm các giá trị C, Q, S
- Không có missing value
  => Ta sẽ fillna bằng giá trị mode và mã hóa cột Embarked dưới dạng số: "C": 1, "Q": 2, "S": 3


In [67]:
cls_embarked = {'C': 1, 'Q': 2, 'S': 3}

In [68]:
df_output['Embarked'] = df_train['Embarked'].fillna(df_train['Embarked'].mode()[0]).apply(lambda x: cls_embarked[x])

## Name: Tên hành khách


In [69]:
np.unique(df_train['Name'].dropna().values)

array(['Abbing, Mr. Anthony', 'Abbott, Mr. Rossmore Edward',
       'Abbott, Mrs. Stanton (Rosa Hunt)', 'Abelson, Mr. Samuel',
       'Abelson, Mrs. Samuel (Hannah Wizosky)',
       'Adahl, Mr. Mauritz Nils Martin', 'Adams, Mr. John',
       'Ahlin, Mrs. Johan (Johanna Persdotter Larsson)',
       'Aks, Mrs. Sam (Leah Rosen)', 'Albimona, Mr. Nassef Cassem',
       'Alexander, Mr. William', 'Alhomaki, Mr. Ilmari Rudolf',
       'Ali, Mr. Ahmed', 'Ali, Mr. William',
       'Allen, Miss. Elisabeth Walton', 'Allen, Mr. William Henry',
       'Allison, Master. Hudson Trevor', 'Allison, Miss. Helen Loraine',
       'Allison, Mrs. Hudson J C (Bessie Waldo Daniels)',
       'Allum, Mr. Owen George',
       'Andersen-Jensen, Miss. Carla Christine Nielsine',
       'Anderson, Mr. Harry', 'Andersson, Master. Sigvard Harald Elias',
       'Andersson, Miss. Ebba Iris Alfrida',
       'Andersson, Miss. Ellis Anna Maria',
       'Andersson, Miss. Erna Alexandra',
       'Andersson, Miss. Ingeborg Con

Nhận xét:

- Cột Name là thuộc tính text
- Không có missing value
  => Ta sẽ tạm thời bỏ cột khỏi tập feature này vì khó xử lý


## SibSp: Anh chị em


In [70]:
np.unique(df_train['SibSp'].dropna().values)

array([0, 1, 2, 3, 4, 5, 8])

In [71]:
df_train['SibSp'].isna().sum()

np.int64(0)

Nhận xét:

- Cột SibSp là thuộc tính Category gồm các giá trị số: 0, 1, 2, 3, 4, 5, 8
- Không có missing value
  => Ta sẽ tạm thời giữ nguyên


## Parch: Ba mẹ, con cái


In [72]:
np.unique(df_train['Parch'].dropna().values)

array([0, 1, 2, 3, 4, 5, 6])

In [73]:
df_train['Parch'].isna().sum()

np.int64(0)

Nhận xét:

- Cột Parch là thuộc tính Category gồm các giá trị số: 0, 1, 2, 3, 4, 5, 6
- Không có missing value
  => Ta sẽ tạm thời giữ nguyên


In [74]:
def preprocessing_feature_01(df_data, is_train = True, is_debug = True, **kwargs):
    df_output = pd.DataFrame()

    # # Sex: gioi tinh
    # cls_sex = {'female': 0, 'male' : 1}
    # df_output["Sex"] = df_data["Sex"].apply(lambda x: cls_sex[x])
    # # Age: median
    # df_output["Age"] = df_data["Age"].fillna(df_data["Age"].median())
    # # Fare, Pclass
    # for name in ['Fare', 'Pclass', 'SibSp', 'Parch']:
    #     df_output[name] = df_data[name]
    # # Cabin
    # cls_cabin = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'T':8, 'Z':0}
    # df_output["Cabin"] = df_data['Cabin'].apply(lambda x: cls_cabin['Z'] if pd.isna(x) else cls_cabin[x[0]])
    # # Embarked
    # cls_embarked = {'0': 0, 'C':1, 'Q':2, 'S':3}
    # df_output["Embarked"] =  df_data['Embarked'].apply(lambda x: cls_embarked['0'] if pd.isna(x) else cls_embarked[x])
    # # Surname
    # surnames = ['Capt.', 'Col.', 'Don.', 'Dr.', 'Jonkheer.', 'Lady.', 'Major.', 
    #         'Master.', 'Miss.', 'Mlle.', 'Mme.', 'Mr.', 'Mrs.', 'Ms.', 'Rev.', 'Sir.', 'the']
    # cls_surnames = dict(zip(surnames, range(len(surnames))))
    # df_output["Surname"] = df_data['Name'].apply(lambda x: cls_surnames[x.split(',')[1].split(' ')[1]])

    # Sex: gioi tinh
    df_output["Sex"] = df_data["Sex"].apply(lambda x: cls_sex[x])
    # Age: median
    df_output["Age"] = df_data["Age"].fillna(df_data["Age"].mean())
    # Fare
    df_output['Fare'] = df_data['Fare'].fillna(df_data['Fare'].mean())
    # Pclass, SibSp, Parch
    for name in ['Pclass', 'SibSp', 'Parch', 'PassengerId']:
        df_output[name] = df_data[name]
    # Cabin
    # Drop, không xử lý
    # Embarked
    cls_embarked = {'C':1, 'Q':2, 'S':3}
    df_output['Embarked'] = df_data['Embarked'].fillna(df_data['Embarked'].mode()[0]).apply(lambda x: cls_embarked[x])
    # Name
    # Drop, không xử lý
    if is_train:
        df_output["Output"] = df_data["Survived"]

    # display.display(df_output)

    if is_debug:
        print("head(10)")
        display.display(df_data.head(5))
        print("tail(10)")
        # display.display(df_data.tail(5))
        print("isna")
        display.display(df_data.isna().sum())
        # Sex: gioi tinh
        print("sex")
        display.display(np.unique(df_data['Sex'], return_counts=True))
        # Age: lay median
        print(f'Age IsNa: {df_data["Age"].isna().sum()}')
        print(f"Age Median: {df_data['Age'].median()}")
        # Fare
        display.display(df_data["Fare"].describe())
        # Cabin
        print("-*10", "Cabin")
        display.display(np.unique(df_data['Cabin'].apply(
            lambda x: 'Z0' if pd.isna(x) else x), return_counts=True))
        # Embarked
        display.display(
            np.unique(df_data['Embarked'].apply(lambda x: '0' if pd.isna(x) else x), return_counts=True)
        )
        globals().update(**locals())
    
    return df_output, None
    pass

# df_train = pd.read_csv(f'{data_dir}/train.csv')
# preprocessing_feature_01(df_train)

# Train


# Main


In [75]:
# def main_feat01(**kwargs):
#     # load data
#     df_train = pd.read_csv(f'{data_dir}/train.csv')
#     df_test = pd.read_csv(f'{data_dir}/test.csv')
#     # preprocessing
#     df_output_train, _ = preprocessing_feature_01(df_train, is_train = True, is_debug = False)
#     df_output_test, _ = preprocessing_feature_01(df_test, is_train = False, is_debug = False)
    
#     # saving
#     os.makedirs(save_dir, exist_ok = True)
    
#     np.savez(f'{save_dir}/data.npz', train=df_output_train, test=df_output_test)

#     kwargs.get('global_cfg', {}).update(**locals())
#     pass

# if params_cfg["action"] == "train_feat01":
#     print("Runing ... [train_feat01]")
#     main_feat01(global_cfg = globals())

def main_feat01(**kwargs):
    # load data
    df_train = pd.read_csv(f'{data_dir}/train.csv')
    df_test = pd.read_csv(f'{data_dir}/test.csv')
    # preprocessing
    df_output_train, _ = preprocessing_feature_01(df_train, is_train=True, is_debug=False)
    df_output_test, _ = preprocessing_feature_01(df_test, is_train=False, is_debug=False)
    
    # saving
    os.makedirs(save_dir, exist_ok=True)
    
    # Lưu trực tiếp DataFrame objects (cần allow_pickle=True khi load)
    np.savez(f'{save_dir}/data.npz', 
             train_data=df_output_train.values,
             test_data=df_output_test.values,
             train_columns=df_output_train.columns.values,  # Lưu tên cột train
             test_columns=df_output_test.columns.values,
             allow_pickle=True)
    
    print("Đã lưu DataFrame với đầy đủ thông tin cột")

    kwargs.get('global_cfg', {}).update(**locals())

if params_cfg["action"] == "train_feat01":
    print("Runing ... [train_feat01]")
    main_feat01(global_cfg = globals())

Runing ... [train_feat01]
Đã lưu DataFrame với đầy đủ thông tin cột


# End
